# 2 Context-Free Languages

----
## 2.1 Context-free Grammars

### Formal definition of a context-free grammar

$G = (V,\Sigma,R,S)$

$\Gamma = V\cup\Sigma$

$u\Rightarrow w$라는 derivation은
$u=v_1 A v_2$ 에 규칙 $A\to w \in R$를 적용해 $v=v_1 w v_2$를 얻을 수 있음을 뜻한다.
($u,v,w,v_1,v_2\in \Gamma^*$, $A\in V$)

$G$가 만들어내는 언어는 $\{ w \mid w\in\Sigma^*, S\Rightarrow^*w \}$

### Examples of context-free grammars

### Designing context-free grammars

### Ambiguity


In [1]:
type Gamma = String
type GString = [Gamma]

newtype CFG = CFG ([Gamma],[Gamma],[Rule],Gamma) deriving Show

data Rule = Gamma :-> GString deriving (Eq,Show)

In [2]:
-- 교과서에 나오는 G3 문법
cfgG3 = CFG (["S"],["a","b"],rs,"S")
  where
  rs = [ "S" :-> ["a","S","b"]
       , "S" :-> ["S","S"]
       , "S" :-> []
       ]

In [3]:
import Data.List

zip (inits [1,2,3,4]) (tails [1,2,3,4])

[([],[1,2,3,4]),([1],[2,3,4]),([1,2],[3,4]),([1,2,3],[4]),([1,2,3,4],[])]

In [4]:
step (CFG(vs,as,rs,_)) gs = -- any step
  [ gsL++gs'++gsR | (gsL, v : gsR) <- zip (inits gs) (tails gs),
                    v `elem` vs, -- v 가 variable 이기만 하면 됨
                    (v' :-> gs') <- rs, v==v' ]

lstep (CFG(vs,as,rs,_)) gs = -- leftmost step
  [ gsL++gs'++gsR | (gsL, v : gsR) <- zip (inits gs) (tails gs),
                    all (`elem` as) gsL, v `elem` vs, -- v는 가장 왼쪽의 variable
                    (v' :-> gs') <- rs, v==v' ]

In [5]:
mapM_ print $ step cfgG3 ["a","S","S","b"]

["a","a","S","b","S","b"]
["a","S","S","S","b"]
["a","S","b"]
["a","S","a","S","b","b"]
["a","S","S","S","b"]
["a","S","b"]

In [6]:
mapM_ print $ lstep cfgG3 ["a","S","S","b"]

["a","a","S","b","S","b"]
["a","S","S","S","b"]
["a","S","b"]

In [7]:
-- derivation step operator
infixl 5 .=> 
(.=>) :: [GString] -> GString -> [GString]
gss .=> gs = gss++[gs]

derivation = (:[])

-- valid derivation judgement
infix 4 |-
(|-) :: CFG -> [GString] -> Bool
cfg |- gss
 | length gss < 2 = error "derivation must have at least one step"
 | otherwise = and [gs2 `elem` step cfg gs1 | (gs1,gs2) <- zip gss (tail gss)]
 
 -- valid leftmost derivation judgement
infix 4 ||-
(||-) :: CFG -> [GString] -> Bool
cfg ||- gss
 | length gss < 2 = error "derivation must have at least one step"
 | otherwise = and [gs2 `elem` lstep cfg gs1 | (gs1,gs2) <- zip gss (tail gss)]

In [8]:
derivation ["S"] .=> ["a","S","b"] .=> ["a","b"]

[["S"],["a","S","b"],["a","b"]]

In [9]:
cfgG3 |- derivation ["S"]

: 

In [10]:
cfgG3 |- derivation ["S"] .=> ["a","S","b"]
cfgG3 |- derivation ["S"] .=> ["a","S","b"] .=> ["a","b"]
cfgG3 |- derivation ["S"] .=> ["a","S","b"] .=> ["a","S","b"]
cfgG3 |- derivation ["S"] .=> ["a","S","b"] .=> ["a","S","S","b"]

True

True

False

True

In [11]:
cfgG3 |- derivation ["S"] .=> ["a","S","b"]
                          .=> ["a","S","S","b"]
                          .=> ["a","S","a","S","b","b"]
                          
cfgG3||- derivation ["S"] .=> ["a","S","b"]
                          .=> ["a","S","S","b"]
                          .=> ["a","S","a","S","b","b"]

True

False

In [12]:
-- G3 is an ambiguous CFG because multiple leftmost derivation exists for S =>* ab
cfgG3||- derivation ["S"] .=> ["a","S","b"]
                          .=> ["a","b"]

cfgG3||- derivation ["S"] .=> ["a","S","b"]
                          .=> ["a","S","S","b"]
                          .=> ["a","S","b"]
                          .=> ["a","b"]

True

True

In [13]:
-- 그냥 parse tree가 유일하면 ambiguous하지 않음
import Data.Tree
-- data Tree a = Node a [a] -- 첫번째 rootLabel 두번째 subForest

tree1 =
  Node "S" [ Node "a" [] -- terminal은 자녀 없음
           , Node "S" [] -- epsilon
           , Node "b" [] -- terminal은 자녀 없음
           ]

putStrLn $ drawTree tree1

S
|
+- a
|
+- S
|
`- b

In [14]:
tree2 =
  Node "S" [ Node "a" [] -- terminal은 자녀 없음
           , Node "S" [ Node "S" [] -- epsilon
                      , Node "S" [] -- epsilon
                      ]
           , Node "b" [] -- terminal은 자녀 없음
           ]

putStrLn $ drawTree tree2

S
|
+- a
|
+- S
|  |
|  +- S
|  |
|  `- S
|
`- b

### Chomsky normal form
CFG 규칙에 $A \to BC$와 $A\to a$의 형태($B$와 $C$는 시작 변수 아님)만 있으면 촘스키정규형(CNF)이라 한다.<br>
단, $\varepsilon$을 포함하는 언어 정의도 가능해야 하므로, 예외적으로 시작 변수 $S$에 대해서만 $S\to \varepsilon$ 규칙을 허용한다.

모든 문맥자유언어(CFL)를 CNF로 나타낼 수 있다는 성질이 알려져 있다.<br>
이는 모든 문맥자유문법(CFG)을 똑같은 언어를 생성하는 CNF로 변환 가능함을 보임으로써 증명 가능하다.

기본적인 아이디어는

 * $A\to \varepsilon$ 형태의 $\varepsilon$규칙과 $A\to B$ 형태의 unit규칙을 제거하고 
 * 그런 형태의 규칙들이 만들어내던 문자열을 CNF를 만족하는 다른 규칙들로 만들어내도록

대체하는 방법만 찾아내면 된다.
그 외에 좀더 고치기 쉬운 형태의 규칙들,<br>
예를 들면 $A \to B_1 B_2 C_1 C_2$는
새로운 변수 $B_0$와 $C_0$를 도입해 아래 규칙들로 대체하면 된다.

  $A\to B_0C_0$<br>
  $B_0\to B_1 B_2$<br>
  $C_0\to C_1 C_2$

CNF가 이론을 전개하고 특정 알고리듬을 설명하는 데 요긴하게 쓰인다.<br>
그러나 CNF를 최소화된 혹은 최적화된 문법규칙이라 말할 수는 없다.<br>
CFG에는 최소화된 문법이라는 개념이 일반적으로 잘 정의되지 않는다.<br>
(참고로, 결정적유한오토마타(DFA)의 경우에는 상태 개수라는 명확한 기준으로 최소화된 DFA로 변환하는 것이 가능)

----
## 2.2 Pushdown Automata

### Pushdown Automata

### Formal definition of a pushdown automaton

### Examples of pushdown automata

### Equivalence with context-free grammars

----
## Non-context-free Languages

### The pumping lemma for context-free languages

----
## Deterministic Context-Free Languages

### Deterministic Context-Free Languages

### Properties of DCFLs

### Deterministic context-free grammars

### Relationship of DPDAs and DCFGs

### Parsing and LR(k) Grammars